In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
def process_company(company_data):
    try:
        m = company_data.money_raised_usd.values
        p = company_data.post_money_valuation_usd.values
        e = company_data.exit_value.values[0]

        if company_data.exit_type.values[0] == 3:
            ret = np.zeros_like(company_data.uuid) - 1
        elif company_data.shape[0] == 1:
            ret = [(m[0] / p[0] * e - m[0]) / m[0]]
        else:
            x0 = m / p # stake of investors at their first round
            x = (p[1:] - m[1:]) / p[1:] # dilution factors
            d = np.cumprod(x)[::-1]
            d = np.insert(d, len(d), 1)
            ret = (e * x0 * d - m) / m
        return pd.DataFrame(data=ret, index=company_data.uuid)
    except:
        print(company_data)
        return pd.DataFrame(data=[np.nan] * company_data.uuid.size, index=company_data.uuid)


def filter_dates(funding_round):
    return funding_round['announced_on'] < funding_round['exit_date']

In [16]:
predicted = False

In [17]:
if predicted:
    rounds_data = pd.read_csv('../../data/obs_predicted_data.csv')
else:
    rounds_data = pd.read_csv('../../data/no_pred_data.csv')

In [18]:
rounds_data

,Unnamed: 0,uuid,org_uuid,announced_on,money_raised_usd,num_investors,investment_type,lead_investor_rank,post_money_valuation_usd,prev_rounds,country_code,amount_delta,holding_time
0,176213,dddc0961-4ac7-42ae-3b70-3ebf960b7ba4,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,2888784000,8.260736e+07,1.0,other,92097.0,NaN,0.0,DEU,8.260736e+07,8963.0
1,143656,e2bc8d8c-7196-0017-3064-60d8b2c80b24,00002470-bff7-6226-5800-0ca1b3787b6f,2870208000,1.600000e+06,1.0,seed,110671.0,NaN,0.0,USA,1.600000e+06,1443.0
2,95828,f1e966a7-06d3-f771-fba4-a613a9fbbdb3,000095de-8e2b-82f1-32a7-c222ba3d5682,2380752000,9.500000e+06,1.0,other,183283.0,NaN,0.0,USA,9.500000e+06,161.0
3,472145,aa40a45e-a0bc-455b-85de-125bbc8adc5e,00011be5-d105-eaa0-69d6-6064e74916b2,3036614400,3.667312e+06,8.0,seed,140301.0,NaN,0.0,USA,3.667312e+06,3326.0
4,213030,cf936930-f849-4d9a-a007-67cc406ca59f,0001eae7-077d-4d0b-a717-f67bcf2a09fa,2917814400,1.275000e+09,1.0,debt_financing,723.0,NaN,0.0,USA,1.275000e+09,1264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122870,333597,4ed08f61-2001-4458-a29e-a651bb1e18be,fffe3b0b-8545-4414-9c0e-a3f961c4e420,2874960000,1.452327e+07,1.0,other,295238.0,NaN,0.0,CHN,1.452327e+07,1450.0
122871,359782,b6cd618e-c9f0-4410-9f5e-4243b2b65fcd,ffff2bea-7e44-4563-b311-65daadb683cd,2680214400,4.514070e+05,1.0,angel,2819467.0,NaN,0.0,CHN,4.514070e+05,207.0
122872,359783,f5cac82f-41b8-4992-abee-ac609fad4e3f,ffff2bea-7e44-4563-b311-65daadb683cd,2755382400,1.616916e+06,1.0,series_a,3439.0,NaN,1.0,CHN,1.165509e+06,870.0
122873,131959,d5d0d173-9a6f-4359-f4bb-a7e71f20b988,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2851804800,2.100000e+05,4.0,seed,43.0,NaN,0.0,USA,2.100000e+05,500.0


In [19]:
orgs = pd.read_csv("../../data/crunchbase/organizations.csv")
acqs = pd.read_csv("../../data/crunchbase/acquisitions.csv")
ipos = pd.read_csv("../../data/crunchbase/ipos.csv")

In [20]:
ipos = ipos[['org_uuid', 'went_public_on', 'valuation_price_usd']]
acqs = acqs[['acquiree_uuid', 'acquired_on', 'price_usd']]

In [21]:
rounds = rounds_data[['uuid', 'org_uuid', 'announced_on', 'money_raised_usd', 'post_money_valuation_usd']].copy()

ipos.columns = ['org_uuid', 'exit_date', 'exit_value']
acqs.columns = ['org_uuid', 'exit_date', 'exit_value']

clos = pd.DataFrame(orgs[orgs.status == 'closed'][['uuid', 'closed_on']])
clos.columns = ['org_uuid', 'exit_date']

ipos["exit_type"] = 1
acqs["exit_type"] = 2
clos["exit_type"] = 3
clos["exit_value"] = np.nan

exits = pd.concat([acqs, ipos, clos]).set_index("org_uuid")

In [22]:
rounds["exit_date"] = pd.to_datetime(rounds.org_uuid.map(exits["exit_date"].to_dict()))
rounds["exit_type"] = rounds.org_uuid.map(exits["exit_type"].to_dict()).fillna(value=4).astype(int)
rounds["exit_value"] = rounds.org_uuid.map(exits["exit_value"].to_dict())

In [23]:
ref_date = '1926-01-01'
rounds.announced_on = pd.Timestamp(ref_date) + rounds.announced_on * pd.Timedelta('1s')
rounds = rounds[(rounds.announced_on < rounds.exit_date)]
rounds = rounds.sort_values(by=['org_uuid', 'announced_on'])

In [24]:
rounds['return_usd'] = rounds.uuid.map(rounds.groupby('org_uuid').apply(process_company).droplevel(0)[0]) + 1

In [25]:
rounds = rounds.drop(columns=['uuid'])
rounds.insert(1, 'group_num', np.nan)
rounds.insert(2, 'seg_num', np.nan)

rounds.columns = ['company_num', 'group_num', 'seg_num', 'round_date', 'raised_usd',
                  'postvalue_usd', 'exit_date', 'exit_type', 'exit_value', 'return_usd']
rounds = rounds[rounds.round_date > '1990-01-01']

In [26]:
operating = rounds_data[['org_uuid', 'announced_on', 'money_raised_usd', 'post_money_valuation_usd']].copy()
operating = operating[~operating.org_uuid.isin(rounds.company_num)]

operating.announced_on = pd.Timestamp(ref_date) + operating.announced_on * pd.Timedelta('1s')
operating.insert(1, 'group_num', np.nan)
operating.insert(2, 'seg_num', np.nan)
operating.insert(operating.shape[1], 'exit_date', np.nan)
operating.insert(operating.shape[1], 'exit_type', 4)
operating.insert(operating.shape[1], 'exit_value', np.nan)
operating.insert(operating.shape[1], 'return_usd', np.nan)

operating.columns = ['company_num', 'group_num', 'seg_num', 'round_date', 'raised_usd',
                  'postvalue_usd', 'exit_date', 'exit_type', 'exit_value', 'return_usd']
operating = operating[operating.round_date.dt.year > 1990]

In [27]:
complete = pd.concat([rounds, operating], ignore_index=True)
complete.loc[complete.return_usd == 0, 'return_usd'] = np.nan
complete = complete[(complete.return_usd < 3000) | (complete.return_usd.isna())]
complete.exit_date = pd.to_datetime(complete.exit_date, errors="coerce")

In [28]:
if predicted:
    complete.to_csv('../../data/data.csv', index=False)
else:
    complete.to_csv('../../data/data_nopred_full.csv', index=False)